In [7]:
import torch
from torch import nn, optim
from torchsummary import summary
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

Device: cuda:0


In [9]:
batch_size_train = 64
batch_size_test = 64

train_data = torchvision.datasets.CIFAR10(root = './data',
                                          train = True,
                                          download=True,
                                          transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root = './data',
                                          train = False,
                                          download=True,
                                          transform=torchvision.transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_data,batch_size = batch_size_train, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size_test, shuffle = True)
print(train_loader)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3*32*32, 1024),
    nn.ReLU(),
    nn.Dropout(p = 0.2),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(p = 0.5),
    nn.Linear(512,10),
    nn.Softmax(-1)
)
model.to(device)
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                 [-1, 3072]               0
            Linear-2                 [-1, 1024]       3,146,752
              ReLU-3                 [-1, 1024]               0
           Dropout-4                 [-1, 1024]               0
            Linear-5                  [-1, 512]         524,800
              ReLU-6                  [-1, 512]               0
           Dropout-7                  [-1, 512]               0
            Linear-8                   [-1, 10]           5,130
           Softmax-9                   [-1, 10]               0
Total params: 3,676,682
Trainable params: 3,676,682
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 14.03
Estimated Total Size (MB): 14.10
-------------------------------------

In [11]:
optimizer = optim.SGD(model.parameters(), momentum = 0.9, lr = 0.001, weight_decay=1e-5)
loss_func = nn.CrossEntropyLoss(reduction = 'mean')
num_epochs = 100

In [12]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, labels)
            test_loss += loss.item()
    print(
        f"Epoch {epoch + 1}/{num_epochs} - Training Loss: {train_loss / len(train_loader):.4f} - Validation Loss: {test_loss / len(test_loader):.4f}")


Epoch 1/100 - Training Loss: 2.3004 - Validation Loss: 2.2971
Epoch 2/100 - Training Loss: 2.2933 - Validation Loss: 2.2871
Epoch 3/100 - Training Loss: 2.2808 - Validation Loss: 2.2691
Epoch 4/100 - Training Loss: 2.2589 - Validation Loss: 2.2420
Epoch 5/100 - Training Loss: 2.2335 - Validation Loss: 2.2170
Epoch 6/100 - Training Loss: 2.2132 - Validation Loss: 2.1993
Epoch 7/100 - Training Loss: 2.1994 - Validation Loss: 2.1867
Epoch 8/100 - Training Loss: 2.1885 - Validation Loss: 2.1782
Epoch 9/100 - Training Loss: 2.1807 - Validation Loss: 2.1704
Epoch 10/100 - Training Loss: 2.1745 - Validation Loss: 2.1661
Epoch 11/100 - Training Loss: 2.1695 - Validation Loss: 2.1625
Epoch 12/100 - Training Loss: 2.1638 - Validation Loss: 2.1563
Epoch 13/100 - Training Loss: 2.1598 - Validation Loss: 2.1524
Epoch 14/100 - Training Loss: 2.1550 - Validation Loss: 2.1476
Epoch 15/100 - Training Loss: 2.1508 - Validation Loss: 2.1439
Epoch 16/100 - Training Loss: 2.1461 - Validation Loss: 2.1391
E

KeyboardInterrupt: 

In [13]:
model.eval()
test_acc = 0
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    predictions = model(images)
    accuracy = (torch.max(predictions, dim=-1, keepdim=True)[1].flatten() == labels).sum() / len(labels)
    test_acc += accuracy.item()
test_acc /= len(test_loader)
print(f"Test accuracy: {test_acc:.3f}")

Test accuracy: 0.399
